In [14]:
from cdcqr.data.deribit.config import get_deribit_sample_data_url
from cdcqr.data.deribit.data_utils import url_csv_zip_file_reader
from cdcqr.analytics.derivatives.option_utils import euro_vanilla
from cdcqr.analytics.derivatives.option_utils import find_vol
from cdcqr.common.config import SECONDS_PER_DAY
from cdcqr.analytics.derivatives.option_utils import euro_vanilla, Put
import cdcqr.data.deribit.data_utils as datautils


## Load Deribit sample data

In [7]:
url = get_deribit_sample_data_url('OPTIONS', 'options_chain')
df = url_csv_zip_file_reader(url)

loading cached Deribit_OPTIONS_options_chain_2020-07-01_2020-07-31.csv.gz
file size > 500MB, load top 5000 rows


In [17]:
df.head(2).T

exchange,deribit,deribit
symbol,BTC-3JUL20-9500-C,BTC-3JUL20-9250-P
timestamp,1593561597178000,1593561598001000
local_timestamp,1593561600161993,1593561600161994
type,call,put
strike_price,9500,9250
expiration,1593763200000000,1593763200000000
open_interest,1186.0,785.0
last_price,0.0035,0.0205
bid_price,0.003,0.0195
bid_amount,0.7,0.3


In [18]:
df1 = df.pipe(datautils.DeribitUtils.parse_optSymbol_col).pipe(datautils.DeribitUtils.parse_time_col)

In [19]:
all_symbols = list(df1['symbol'].unique())
all_symbols

['BTC-3JUL20-9500-C',
 'BTC-3JUL20-9250-P',
 'BTC-3JUL20-8250-P',
 'BTC-3JUL20-7750-P',
 'BTC-3JUL20-6750-C',
 'BTC-3JUL20-12250-P',
 'BTC-3JUL20-12250-C',
 'BTC-3JUL20-12000-P',
 'BTC-3JUL20-12000-C',
 'BTC-3JUL20-11250-P',
 'BTC-3JUL20-10250-C',
 'BTC-31JUL20-9250-C',
 'BTC-31JUL20-8000-C',
 'BTC-31JUL20-5500-C',
 'BTC-31JUL20-5000-C',
 'BTC-31JUL20-18000-C',
 'BTC-31JUL20-5000-P',
 'BTC-31JUL20-15000-P',
 'BTC-31JUL20-14000-C',
 'BTC-31JUL20-10500-P',
 'BTC-31JUL20-10500-C',
 'BTC-2JUL20-9750-P',
 'BTC-2JUL20-9375-C',
 'BTC-2JUL20-8750-P',
 'BTC-2JUL20-8375-C',
 'BTC-2JUL20-8250-P',
 'BTC-2JUL20-8250-C',
 'BTC-2JUL20-10000-P',
 'BTC-28AUG20-9000-P',
 'BTC-28AUG20-15000-P',
 'BTC-28AUG20-14000-C',
 'BTC-28AUG20-13000-P',
 'BTC-28AUG20-12000-P',
 'BTC-28AUG20-10500-P',
 'BTC-26MAR21-9000-P',
 'BTC-26MAR21-8000-P',
 'BTC-26MAR21-6000-P',
 'BTC-26MAR21-4000-P',
 'BTC-26MAR21-32000-C',
 'BTC-26MAR21-28000-C',
 'BTC-26MAR21-20000-P',
 'BTC-26MAR21-11000-P',
 'BTC-26MAR21-11000-C',
 'BTC-2

In [10]:
dates = sorted(list(set([x.split('-')[1] for x in all_symbols])))

In [21]:
#df['timestamp_s'], df['microsec'] = zip(*df['timestamp'].apply(DataTimeUtils.epoch_time_converter, 1))
#df['exp_timestamp_s'], df['exp_microsec'] = zip(*df['expiration'].apply(DataTimeUtils.epoch_time_converter, 1))

In [22]:
df1['underlying_index'].unique()

array(['SYN.BTC-3JUL20', 'SYN.BTC-31JUL20', 'SYN.BTC-2JUL20',
       'SYN.BTC-28AUG20', 'BTC-26MAR21', 'BTC-25SEP20', 'BTC-25DEC20',
       'SYN.BTC-1JUL20', 'SYN.BTC-10JUL20', 'SYN.ETH-3JUL20',
       'SYN.ETH-31JUL20', 'SYN.ETH-2JUL20', 'SYN.ETH-28AUG20',
       'ETH-26MAR21', 'ETH-25SEP20', 'ETH-25DEC20', 'SYN.ETH-1JUL20',
       'SYN.ETH-10JUL20'], dtype=object)

In [25]:
opt1 = df.iloc[1,:]
opt1

symbol              BTC-3JUL20-9250-P
timestamp            1593561598001000
local_timestamp      1593561600161994
type                              put
strike_price                     9250
expiration           1593763200000000
open_interest                   785.0
last_price                     0.0205
bid_price                      0.0195
bid_amount                        0.3
bid_iv                          38.49
ask_price                      0.0225
ask_amount                        0.3
ask_iv                          48.45
mark_price                   0.022261
mark_iv                         47.63
underlying_index       SYN.BTC-3JUL20
underlying_price              9137.12
delta                        -0.61919
gamma                         0.00109
vega                          2.78342
theta                       -28.40972
rho                          -0.37467
Name: deribit, dtype: object

### Verify BS formula

In [10]:
T = (opt1['expiration'] - opt1['timestamp'])/(1e6*SECONDS_PER_DAY*365)
T

0.006392757451801116

In [11]:
target_value = opt1['bid_price']
S = opt1['underlying_price']
K = opt1['strike_price']
sigma = opt1['ask_iv']/100
r = 0 

In [12]:
print('S={},'.format(S), 'K={},'.format(K), 'T={},'.format(T), 'r={},'.format(r), 'sgima={},'.format(sigma))

S=9137.12, K=9250, T=0.006392757451801116, r=0, sgima=0.48450000000000004,


In [13]:
put_ask_iv = euro_vanilla(S, K, T, r, sigma, option='put')

In [14]:
put_ask_iv, opt1['ask_price']*opt1['underlying_price']

(205.58690614397256, 205.58520000000001)

### chceck greeks using opstrat

In [15]:
Put.Price?

Signature: Put.Price(S, K, r, sigma, T)
Docstring: <no docstring>
File:      c:\users\wang\documents\github\croqr\croqr\analytics\options\option_utils.py
Type:      function


In [16]:
Put.Price(S, K, r, sigma, T)

205.58690614397256

In [17]:
Put.Delta(S, K, r, sigma, T)

-0.6169913099694828

In [18]:
Put.Gamma(S, K, r, sigma, T)

0.0010782824572031426

In [22]:
Put.Vega(S, K, r, sigma, T)/100

2.788259595507463

In [27]:
Put.Theta(S, K, r, sigma, T)/365

-28.94782240617039

In [26]:
Put.Rho(S, K, r, sigma, T)/100

-0.3735358847372249

In [ ]:
delta                          -0.61919
gamma                           0.00109
vega                            2.78342
theta                         -28.40972
rho                            -0.37467

### snapshot example from 30SEP2021

In [27]:
bid = 0.0095
ask = 0.0105

IV_bid = 0.854
IV_ask = 0.874
bid_price = 400.75
ask_price = 421.84
S = 42166
K = 60000
r=0
T = 30/365
sigma = IV_bid

In [28]:
print('S={},'.format(S), 'K={},'.format(K), 'T={},'.format(T), 'r={},'.format(r), 'sgima={},'.format(sigma))

S=42166, K=60000, T=0.0821917808219178, r=0, sgima=0.854,


In [29]:
call_price_IV_bid = euro_vanilla(S, K, T, r, IV_bid, option='call')
print('call price using IV bid ({}) =  {} '.format(IV_bid, call_price_IV_bid))
print('call bid price {}'.format(bid_price))

call_price_IV_ask = euro_vanilla(S, K, T, r, IV_ask, option='call')
print('call price using IV ask ({}) =  {} '.format(IV_ask, call_price_IV_ask))
print('call ask price {}'.format(ask_price))

call price using IV bid (0.854) =  410.39147470840635 
call bid price 400.75
call price using IV ask (0.874) =  451.812848334489 
call ask price 421.84


In [116]:
IV_bid = find_vol(bid_price, S, K, T, r)
IV_ask = find_vol(ask_price, S, K, T, r)

In [117]:
IV_bid, IV_ask

(0.8492054360526142, 0.8596221403404919)